# GRUANpy Proof of Concept

GRUANpy is a python toolkit to facilitate the analysis of GRUAN data. It includes several functionalities and is easily extensible thanks to its structure based on the inclusion of different data models and helper classes that provide specialized methods for different types of purposes. The different functions can also be executed in succession in order to create real data pipelines that allow a large variety of outputs.

Here is a list of the main features identified so far:
- download, the ability to download data of interest
- merge, the ability to merge data from different products
- aggregation, the ability to aggregate different observations and lower the resolution of the data (respecting the GRUAN principles in uncertainty processing)

In [1]:
# Demo setup
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from gruanpy import GRUANpy
from datetime import datetime
download_folder=r"gdp_demo_090625"
gp = GRUANpy(download_folder=download_folder)
%matplotlib qt


## DOWNLOAD

In [ ]:
# Download Gruan Data Product (GDP) through NOAA FTP
ftp_dir_path=r'pub/data/gruan/processing/level2/RS92-GDP/version-002/LIN/2018'
files=gp.search(ftp_dir_path=ftp_dir_path)
print(f"Found {len(files)} files in {ftp_dir_path}")
for file in files[:5]:
    print(file)
print("-----"*10)
print("Downloading first 2 files for demo purpose")
files=files[:2]
for file in files:
    gp.download(ftp_dir_path=ftp_dir_path, file_name=file)
print("Download completed.")
print("-----"*10)
for file in files:
    gdp=gp.read(download_folder+r'/ '[:-1]+file)
    print(gdp.global_attrs.head())
    print("-----"*10)

# Iterative script at code_examples\download_gdp.py

Found 75 files in pub/data/gruan/processing/level2/RS92-GDP/version-002/LIN/2018
LIN-RS-01_2_RS92-GDP_002_20180611T180000_1-002-001.nc
LIN-RS-01_2_RS92-GDP_002_20180103T000000_1-002-002.nc
LIN-RS-01_2_RS92-GDP_002_20180612T002400_1-000-002.nc
LIN-RS-01_2_RS92-GDP_002_20180122T120000_1-002-001.nc
LIN-RS-01_2_RS92-GDP_002_20180613T180000_1-002-002.nc
--------------------------------------------------
Download completed.
--------------------------------------------------
     Attribute                                              Value
0  Conventions                                             CF-1.4
1        title                RS92 GRUAN Data Product (Version 2)
2  institution  MOL - Lindenberg Meteorological Observatory; D...
3       source                                           RS92-SGP
4      history  2018-06-11 21:30:51.000Z RS92-GDP: RS92 GRUAN ...
--------------------------------------------------
     Attribute                                              Value
0  Conventions

In [ ]:
# Download GRUAN DATA through CDS API in csv format

api_response_file = "api_response.csv"
api_request = """
import cdsapi

dataset = "insitu-observations-gruan-reference-network"
request = {
    "variable": [
        "air_temperature",
        "relative_humidity",
        "relative_humidity_effective_vertical_resolution",
        "wind_speed",
        "wind_from_direction",
        "eastward_wind_speed",
        "northward_wind_speed",
        "shortwave_radiation",
        "air_pressure",
        "altitude",
        "geopotential_height",
        "frost_point_temperature",
        "water_vapour_volume_mixing_ratio",
        "vertical_speed_of_radiosonde",
        "time_since_launch"
    ],
    "year": "2014",
    "month": "10",
    "day": ["14"],
    "data_format": "csv",
    "area": [90, 0, 0, 90]
}

client = cdsapi.Client()
"""+f"""
target= r"{download_folder}\\{api_response_file}" # Change this to your desired output path
client.retrieve(dataset, request, target)

"""
print("Executing API request ...")
gp.exec_request(api_request)
print("-----"*10)


Executing API request ...


2025-05-28 10:53:45,454 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-28 10:53:45,455 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-28 10:53:46,171 INFO Request ID is 1cee6683-13e9-4835-83c8-6c0deddbaf89
2025-05-28 10:53:46,275 INFO status has been updated to accepted
2025-05-28 10:55:02,255 INFO status has been updated to successful
                                                                                          

--------------------------------------------------


In [6]:
# Download GRUAN DATA through CDS API in netCDF format

api_response_file = "api_response.nc"
api_request = """
import cdsapi

dataset = "insitu-observations-gruan-reference-network"
request = {
    "variable": [
        "air_temperature",
        "relative_humidity",
        "relative_humidity_effective_vertical_resolution",
        "wind_speed",
        "wind_from_direction",
        "eastward_wind_speed",
        "northward_wind_speed",
        "shortwave_radiation",
        "air_pressure",
        "altitude",
        "geopotential_height",
        "frost_point_temperature",
        "water_vapour_volume_mixing_ratio",
        "vertical_speed_of_radiosonde",
        "time_since_launch"
    ],
    "year": "2014",
    "month": "10",
    "day": ["14"],
    "data_format": "netcdf",
    "area": [90, 0, 0, 90]
}

client = cdsapi.Client()
"""+f"""
target= r"{download_folder}\\{api_response_file}" # Change this to your desired output path
client.retrieve(dataset, request, target)

"""
print("Executing API request ...")
gp.exec_request(api_request)
print("-----"*10)

Executing API request ...


2025-05-28 11:16:44,989 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-28 11:16:44,990 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-28 11:16:45,706 INFO Request ID is f5bfd683-46f5-40fb-bca0-50816aa87b21
2025-05-28 11:16:45,809 INFO status has been updated to accepted
2025-05-28 11:17:35,883 INFO status has been updated to successful
                                                                                         

--------------------------------------------------


# MERGE and AGGREGATE

In [2]:
# GDP RS41 spatial gridding
file_path = r'gdp_demo_090625\LIN-RS-01_2_RS41-GDP_001_20141209T120000_1-009-002.nc'
gdp=gp.read(file_path)

bin_column = 'press' # Choose the binning column (alt or press)
target_columns = ['temp', 'rh']

ggd = gp.spatial_gridding(gdp, bin_column, target_columns)

# Plot original and gridded data
for column in target_columns:
    fig, ax1 = plt.subplots(figsize=(7, 6))
    if bin_column == 'press':
        ax1.set_yscale('log')
        ax1.invert_yaxis()
    ax1.scatter(gdp.data[column], gdp.data[bin_column], label='Original Data', alpha=0.5)
    ax1.scatter(ggd.data[column], ggd.data[bin_column], label='Gridded Data', color='red', alpha=0.5)
    ax1.fill_betweenx(gdp.data[bin_column], gdp.data[column] - gdp.data[column+'_uc'], gdp.data[column] + gdp.data[column+'_uc'], color='blue', alpha=0.2, label='Original Uncertainty')
    ax1.fill_betweenx(ggd.data[bin_column], ggd.data[column] - ggd.data[column+'_uc'], ggd.data[column] + ggd.data[column+'_uc'], color='red', alpha=0.2, label='Gridded Uncertainty')
    ax1.set_xlabel(f'{column.capitalize()} {gdp.variables_attrs[gdp.variables_attrs['variable'] == column]['units'].values[0]}')
    ax1.set_ylabel(f'{bin_column.capitalize()} {gdp.variables_attrs[gdp.variables_attrs['variable'] == bin_column]['units'].values[0]}')
    ax1.legend()
    long_name= gdp.variables_attrs[gdp.variables_attrs['variable'] == column]['long_name'].values[0]
    ax1.set_title(f'{long_name} Mandatory Levels Spatial Gridding')

    plt.tight_layout()
    plt.show()

In [7]:
# GDP RS41 temporal gridding

# Read multiple GDP files
gdp_folder=r'C:\Users\tomma\Documents\SDC\Repos\GRUAN_EDA\gdp\products_RS41-GDP-1_LIN_2017'#gdp_demo_090625\products_RS41-GDP-1_LIN_2017' # Path to the folder
gdp_files = [os.path.join(gdp_folder, f) for f in os.listdir(gdp_folder) if f.endswith('.nc')]
gdps=[]
for file in tqdm(gdp_files[:250] , desc="Reading GDPs"):
    gdps.append(gp.read(file))

Reading GDPs: 100%|██████████| 250/250 [06:31<00:00,  1.57s/it]


In [8]:
# Uniform Spatial gridding accross multiple GDPs
ggds=[]
target_columns = ['temp', 'rh']
for gdp in tqdm(gdps, desc="Spatial Gridding"):
    ggd = gp.spatial_gridding(gdp, 'press', target_columns, mandatory_levels_flag=True)
    ggds.append(ggd)

Spatial Gridding:   0%|          | 0/250 [00:00<?, ?it/s]

Spatial Gridding:  20%|█▉        | 49/250 [00:02<00:10, 19.49it/s]c:\Users\tomma\Documents\SDC\Repos\GRUAN_EDA\gruanpy\helpers\grid\gridding_manager.py:43: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: (((x-x.mean())**2).sum()/(len(x)*(len(x)-1)))**0.5
c:\Users\tomma\Documents\SDC\Repos\GRUAN_EDA\gruanpy\helpers\grid\gridding_manager.py:43: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: (((x-x.mean())**2).sum()/(len(x)*(len(x)-1)))**0.5
Spatial Gridding: 100%|██████████| 250/250 [00:10<00:00, 24.15it/s]


In [9]:
# Merge all gridded data
mggd=pd.DataFrame()
for ggd in tqdm(ggds, desc="Merging Gridded Data"):
    start_time_str = ggd.metadata[ggd.metadata['Attribute'] == 'g.Measurement.StartTime']['Value'].values[0]
    start_time = datetime.strptime(start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    ggd_data = ggd.data.copy()
    ggd_data['time'] = start_time
    mggd = pd.concat([mggd, ggd_data], ignore_index=True)

# Plotting the merged gridded data
for column in target_columns:
    # Separate data into day and night based on time (e.g., 6:00-18:00 as day, else night)
    mand_lvl_val = 1000
    subset = mggd[mggd['mand_lvl'] == mand_lvl_val].copy()
    subset['hour'] = subset['time'].dt.hour
    day_data = subset[(subset['hour'] >= 6) & (subset['hour'] < 18)]
    night_data = subset[(subset['hour'] < 6) | (subset['hour'] >= 18)]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(day_data['time'], day_data[column], marker='o', linestyle='-', label=f'{column} (Day)')
    ax.plot(night_data['time'], night_data[column], marker='x', linestyle='--', label=f'{column} (Night)')
    ax.fill_between(day_data['time'], day_data[column] - day_data[column + '_uc'], day_data[column] + day_data[column + '_uc'], alpha=0.2, label='Day Uncertainty')
    ax.fill_between(night_data['time'], night_data[column] - night_data[column + '_uc'], night_data[column] + night_data[column + '_uc'], alpha=0.2, label='Night Uncertainty')
    ax.set_xlabel('Time')
    ax.set_ylabel(column.capitalize())
    ax.set_title(f'{column.capitalize()} at Mandatory Level {mand_lvl_val} Over Time (Day vs Night)')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Merging Gridded Data: 100%|██████████| 250/250 [00:00<00:00, 776.48it/s]


In [12]:
mggd.describe()

,mand_lvl,temp,rh,press,temp_uc_ucor_avg,temp_var,temp_uc_ucor,temp_uc_scor,temp_uc_tcor,temp_uc,rh_uc_ucor_avg,rh_var,rh_uc_ucor,rh_uc_scor,rh_uc_tcor,rh_uc,time,time_bin,hour
count,4189.000000,4187.000000,4187.000000,4189.000000,4189.000000,4188.000000,4188.000000,4187.000000,4187.000000,4186.000000,4189.000000,4188.000000,4188.000000,4189.0,4187.000000,4186.000000,4189,4189.000000,4189.000000
mean,331.245166,235.224625,27.779970,330.414712,0.004697,0.105435,0.105857,0.055631,0.102232,0.171571,0.053714,0.296527,0.307812,0.0,1.698676,1.760114,2017-04-12 09:35:42.864575,15.709477,13.199093
min,3.000000,193.152588,-0.113309,3.624527,0.000000,0.000000,0.000000,0.000000,0.076148,0.076695,0.000000,0.000000,0.000000,0.0,0.458792,0.458823,2017-01-02 22:45:18.235000,7.000000,0.000000
25%,50.000000,216.489899,1.386320,49.396492,0.001420,0.041047,0.041888,0.000000,0.079894,0.123965,0.012421,0.004967,0.014177,0.0,1.183849,1.196393,2017-03-30 23:12:59.985000,7.000000,10.000000
50%,200.000000,223.307159,6.388488,199.058258,0.003641,0.090774,0.090957,0.050691,0.091424,0.158485,0.033583,0.097143,0.114955,0.0,1.427462,1.468612,2017-04-14 22:47:18.133000,21.000000,10.000000
75%,500.000000,254.745575,54.824768,523.151611,0.006303,0.161966,0.162013,0.087772,0.114724,0.215127,0.081751,0.469683,0.481819,0.0,2.158672,2.286344,2017-05-01 04:45:38.633000,21.000000,22.000000
max,1000.000000,294.477173,102.712555,1006.881653,0.201937,0.472317,0.472745,0.283127,0.883229,0.929598,0.596110,2.694376,2.695872,0.0,5.830813,6.003841,2017-05-16 04:57:13.312000,35.000000,23.000000
std,334.353244,24.349846,32.053711,329.918662,0.006119,0.070184,0.069903,0.050322,0.035354,0.063094,0.053858,0.406141,0.404858,0.0,0.742481,0.773330,NaN,9.456524,6.659905


In [13]:
tggd.describe()

,mand_lvl,time_bin,temp,rh,time,temp_uc_ucor_avg,temp_var,temp_uc_sc,temp_uc_ucor,temp_cor,temp_uc,rh_uc_ucor_avg,rh_var,rh_uc_sc,rh_uc_ucor,rh_cor,rh_uc
count,57.000000,57.000000,57.000000,57.000000,57,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.0,57.000000,57.000000,57.000000
mean,293.157895,21.000000,235.360901,24.251347,2017-05-29 00:00:00,0.023974,0.788816,0.028611,0.801665,0.139161,0.832229,0.055174,1.873185,0.0,1.874858,1.583958,2.720352
min,3.000000,7.000000,213.190811,0.023550,2017-02-20 00:00:00,0.004890,0.000000,0.005699,0.156668,0.088951,0.247275,0.002623,0.000000,0.0,0.005397,0.458792,0.458823
25%,20.000000,7.000000,216.231796,1.352931,2017-02-20 00:00:00,0.010179,0.471126,0.011105,0.471326,0.104766,0.517432,0.004622,0.083826,0.0,0.084149,1.300792,1.304952
50%,150.000000,21.000000,224.903763,2.379170,2017-05-29 00:00:00,0.018442,0.661714,0.015625,0.661883,0.110186,0.682432,0.020745,0.327831,0.0,0.328669,1.432662,1.474606
75%,500.000000,35.000000,252.254410,46.426201,2017-09-04 00:00:00,0.031390,1.040189,0.026235,1.040564,0.117179,1.046073,0.093823,2.781249,0.0,2.782395,2.007960,3.681653
max,1000.000000,35.000000,284.310822,81.328835,2017-09-04 00:00:00,0.077759,2.442241,0.283127,2.443834,0.883229,2.450481,0.201251,8.822773,0.0,8.825068,2.821980,8.971082
std,333.738445,11.532563,23.198839,27.600204,NaN,0.017823,0.480376,0.048745,0.462680,0.129904,0.447023,0.057583,2.288618,0.0,2.288626,0.552382,2.035204


In [15]:
tggd.head()

,mand_lvl,time_bin,temp,rh,time,temp_uc_ucor_avg,temp_var,temp_uc_sc,temp_uc_ucor,temp_cor,temp_uc,rh_uc_ucor_avg,rh_var,rh_uc_sc,rh_uc_ucor,rh_cor,rh_uc
0,3,7.0,253.628830,0.023550,2017-04-10,0.070995,0.000000,0.139659,0.156668,0.191312,0.247275,0.005397,0.000000,0.0,0.005397,0.458792,0.458823
1,3,21.0,239.440292,0.398409,2017-10-23,0.031740,1.596277,0.032852,1.596931,0.207207,1.610317,0.002933,0.069976,0.0,0.070037,0.781120,0.784254
2,3,35.0,239.769226,0.110081,2018-05-07,0.018442,1.190013,0.021036,1.190342,0.139772,1.198520,0.006067,0.055108,0.0,0.055441,0.887708,0.889438
3,5,7.0,239.352478,0.342495,2017-04-10,0.016956,0.941809,0.017435,0.942123,0.125829,0.950488,0.003143,0.057225,0.0,0.057312,1.080691,1.082209
4,5,21.0,233.735733,0.540712,2017-10-23,0.013360,0.779168,0.015355,0.779434,0.117439,0.788232,0.003580,0.070173,0.0,0.070264,1.329574,1.331429


In [22]:
mggd['time'].dt.day

0        2
1        2
2        2
3        2
4        2
        ..
4184    16
4185    16
4186    16
4187    16
4188    16
Name: time, Length: 4189, dtype: int32

In [ ]:
# Temporal gridding from scratch

bin_size = 14 # Size of the time bin in days
first_data = mggd['time'].min()
mggd['time_bin'] = ((mggd['time'].day -first_data.dt.day) // bin_size) * bin_size + bin_size / 2 # WRONG!!! .day take just the day of the month

# Filter mggd to only include rows where the hour is 12 (i.e., time is in the day)
mggd['hour'] = mggd['time'].dt.hour
dmggd = mggd[(mggd['hour'] >= 6) & (mggd['hour'] < 18)] #day merged gridded data
nmggd = mggd[(mggd['hour'] < 6) | (mggd['hour'] >= 18)] #night merged gridded data
fmggd = dmggd #choose focus merged gridded data

tggd = fmggd.groupby(['mand_lvl', 'time_bin'])[target_columns].mean().reset_index() # 3.12
tggd['time'] = pd.to_datetime(tggd['time_bin']*bin_size, unit='D', origin=first_data)
for col in target_columns:
    tggd[col + '_uc_ucor_avg'] = fmggd.groupby(['time_bin','mand_lvl'])[col + '_uc_ucor'].apply(
                lambda x: (((x**2).sum())**0.5)/len(x)
                ).reset_index(drop=True) #3.13
    tggd[col + '_var'] = fmggd.groupby(['time_bin','mand_lvl'])[col].apply(
                lambda x: ((((x-x.mean())**2).sum())/(len(x)*max((len(x)-1),1)))**0.5
                ).reset_index(drop=True) #3.14
    tggd[col + '_uc_sc']=fmggd.groupby(['time_bin','mand_lvl'])[col + '_uc_scor'].apply(
                lambda x: (((x**2).sum())**0.5)/len(x)
                ).reset_index(drop=True) #3.15
    tggd[col + '_uc_ucor']=(
        tggd[col+'_uc_ucor_avg']**2 + tggd[col + '_var']**2 + tggd[col + '_uc_sc']**2)**0.5 #3.16
    tggd[col + '_cor']=fmggd.groupby(['time_bin','mand_lvl'])[col + '_uc_tcor'].mean().reset_index(drop=True) #3.17
    tggd[col+'_uc']=(
        tggd[col+'_uc_ucor']**2 + tggd[col+'_cor']**2)**0.5 #3.18
    
# Plotting the temporal gridded data
for column in target_columns:
    # Separate data into day and night based on time (e.g., 6:00-18:00 as day, else night)
    mand_lvl_val = 1000
    subset = tggd[tggd['mand_lvl'] == mand_lvl_val].copy()
    subset['hour'] = subset['time'].dt.hour
    # Also plot the original day_data for comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(day_data['time'], day_data[column], marker='x', linestyle='--', label=f'{column} (Original Day)')
    ax.fill_between(day_data['time'], day_data[column] - day_data[column + '_uc'], day_data[column] + day_data[column + '_uc'], alpha=0.1, label='Original Day Uncertainty')
    ax.plot(subset['time'], subset[column], marker='o', linestyle='-', label=f'{column} (Temporal Gridded)')
    ax.fill_between(subset['time'], subset[column] - subset[column + '_uc'], subset[column] + subset[column + '_uc'], alpha=0.2, label='Uncertainty')
    ax.set_xlabel('Time')
    ax.set_ylabel(column.capitalize())
    ax.set_title(f'{column.capitalize()} at Mandatory Level {mand_lvl_val} Over Time (Temporal Gridded)')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

AttributeError: 'Series' object has no attribute 'day'

## PIPELINE

# DATA VISUALIZATION